# No-Slope: LOO vs Different Pool Sizes (5 vs 30 batches)

**Goal:** Show that for **no-slope**, different pooling (5 vs 30 batches) gives **similar** AUCs — unlike slope, where pool-39 vs pool-5 made a large difference on the same 100k.

**Data:**
- **No-slope LOO (400k):** Pool 39 per fold, 400k predictions → `results_feb18_old/static_10yr_results_400k.csv`
- **No-slope holdout pool-5 (100k):** Train batches 10–14 (5 batches), test 0–9 → `holdout_auc_slope_1phase_vs_noslope.csv` (noslope rows)
- **No-slope holdout pool-30 (100k):** Train batches 10–39 (30 batches), test 0–9 → `holdout_auc_slope_vs_noslope.csv` (noslope rows)

**Takeaway:** Pool-5 and pool-30 no-slope on the same 100k are very similar; LOO 400k is in the same ballpark. So no-slope is **not** sensitive to pool size, unlike slope.

Then we show **gamma stability** (boxplots) from `gamma_stability_slope_runs.py`: no-slope gamma vs slope gamma_level vs slope gamma_slope across 40 batches.

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

CLAUDE_DIR = Path('/Users/sarahurbut/aladynoulli2/claudefile')
RESULTS_OLD = CLAUDE_DIR / 'results_feb18_old'
RESULTS_HOLDOUT = CLAUDE_DIR / 'results_holdout_auc'

## Load no-slope AUCs: LOO 400k, holdout pool-5 (100k), holdout pool-30 (100k)

In [ ]:
# No-slope LOO on 400k
loo_400k = pd.read_csv(RESULTS_OLD / 'static_10yr_results_400k.csv')
loo_400k = loo_400k.rename(columns={'Disease': 'disease', 'AUC': 'auc_noslope_LOO_400k'})
loo_400k = loo_400k.set_index('disease')['auc_noslope_LOO_400k']

# No-slope holdout pool-5 (1-phase run: train 10-14, test 0-9)
h1 = pd.read_csv(RESULTS_HOLDOUT / 'holdout_auc_slope_1phase_vs_noslope.csv')
noslope_5 = h1[(h1['model'] == 'noslope') & (h1['horizon'] == 'static_10yr')].set_index('disease')['auc']
noslope_5.name = 'auc_noslope_pool5_100k'

# No-slope holdout pool-30 (default: train 10-39, test 0-9)
h2 = pd.read_csv(RESULTS_HOLDOUT / 'holdout_auc_slope_vs_noslope.csv')
noslope_30 = h2[(h2['model'] == 'noslope') & (h2['horizon'] == 'static_10yr')].set_index('disease')['auc']
noslope_30.name = 'auc_noslope_pool30_100k'

compare = pd.DataFrame({'No-slope LOO (400k)': loo_400k, 'No-slope pool-5 (100k)': noslope_5, 'No-slope pool-30 (100k)': noslope_30})
compare = compare.dropna(how='all').sort_values('No-slope LOO (400k)', ascending=False)
compare.head(12)

## Summary: differences between pool-5 and pool-30 on same 100k

In [ ]:
diff_5_30 = (compare['No-slope pool-5 (100k)'] - compare['No-slope pool-30 (100k)']).dropna()
mean_abs_diff = diff_5_30.abs().mean()
print('No-slope: pool-5 vs pool-30 on same 100k test set')
print('='*55)
print(f'  Mean |pool5 − pool30| AUC = {mean_abs_diff:.4f}')
print(f'  Max  |pool5 − pool30|     = {diff_5_30.abs().max():.4f}')
print()
print('No-slope LOO (400k) vs pool-5 (100k): different N, so not direct comparison;')
print('  LOO 400k and pool-5 100k are in the same ballpark (no big jump like slope).')

## Bar plot: No-slope LOO 400k vs pool-5 100k vs pool-30 100k

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(11, 6))
diseases = compare.index.tolist()
x = np.arange(len(diseases))
w = 0.26
ax.bar(x - w, compare['No-slope LOO (400k)'], w, label='No-slope LOO (400k)', color='seagreen', alpha=0.85)
ax.bar(x, compare['No-slope pool-5 (100k)'], w, label='No-slope pool-5 (100k)', color='steelblue', alpha=0.85)
ax.bar(x + w, compare['No-slope pool-30 (100k)'], w, label='No-slope pool-30 (100k)', color='mediumpurple', alpha=0.75)
ax.set_xticks(x)
ax.set_xticklabels(diseases, rotation=45, ha='right')
ax.set_ylabel('Static 10yr AUC')
ax.set_title('No-Slope: Different Pool Sizes Give Similar Results (unlike slope)')
ax.legend(loc='lower right')
ax.axhline(y=0.5, color='gray', linestyle='--', linewidth=0.8)
plt.tight_layout()
out_path = RESULTS_HOLDOUT / 'noslope_loo_vs_pool5_pool30_comparison.pdf'
plt.savefig(out_path, bbox_inches='tight', dpi=150)
plt.show()
print(f'Saved: {out_path}')

## Gamma stability across batches

From `gamma_stability_slope_runs.py`: stability of **no-slope gamma** vs **slope gamma_level** vs **slope gamma_slope** across the same 40 batches (LOO training). Higher pairwise correlation = more stable across batches.

In [ ]:
stab = pd.read_csv(RESULTS_HOLDOUT / 'gamma_stability_slope_runs.csv')
stab = stab[['run', 'param', 'n_batches', 'mean_abs', 'mean_std_across_batches', 'min_pairwise_corr', 'mean_pairwise_corr']]
stab

No-slope gamma has **lower** batch–batch correlation than slope gamma_level (0.58 vs 0.77), yet no-slope AUC was **not** sensitive to pool size. Slope’s extra parameter (gamma_slope) is noisier and drives the need for more batches when using the slope model.

## Boxplots: gamma value across 40 batches (by PRS×Signature cell)

Full boxplots are produced by `gamma_stability_slope_runs.py` and saved as `results_holdout_auc/gamma_stability_boxplot.pdf`. Run the script if the PDF is missing; then view that file. Here we show a small bar chart of mean pairwise correlation (stability summary).

In [ ]:
fig2, ax2 = plt.subplots(figsize=(6, 4))
labels = ['No-slope\ngamma', 'Slope\ngamma_level', 'Slope\ngamma_slope']
vals = [stab[stab['param']=='gamma_noslope']['mean_pairwise_corr'].values[0],
        stab[stab['param']=='gamma_level_1phase']['mean_pairwise_corr'].values[0],
        stab[stab['param']=='gamma_slope_1phase']['mean_pairwise_corr'].values[0]]
colors = ['seagreen', 'steelblue', 'firebrick']
bars = ax2.bar(labels, vals, color=colors, alpha=0.85)
ax2.set_ylabel('Mean pairwise correlation across 40 batches')
ax2.set_title('Gamma stability: higher = more consistent across batches')
ax2.set_ylim(0, 1)
for b, v in zip(bars, vals):
    ax2.text(b.get_x() + b.get_width()/2, b.get_height() + 0.02, f'{v:.3f}', ha='center', fontsize=10)
plt.tight_layout()
plt.savefig(RESULTS_HOLDOUT / 'gamma_stability_summary_bars.pdf', bbox_inches='tight', dpi=150)
plt.show()
print('Full boxplots (per PRS×Sig cell): results_holdout_auc/gamma_stability_boxplot.pdf')
print('(Run: python gamma_stability_slope_runs.py)')

In [ ]:
# Save comparison table
compare.reset_index().to_csv(RESULTS_HOLDOUT / 'noslope_loo_vs_pool5_pool30_comparison.csv', index=False)
print('Saved: results_holdout_auc/noslope_loo_vs_pool5_pool30_comparison.csv')